In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import openai
import os

import warnings
warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

# 시각화 한글폰트 설정
plt.rc('font', family='Malgun')
sns.set(font="Malgun Gothic",
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.
import joblib
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Flatten, SimpleRNN, LSTM, GRU
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
import joblib

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

In [ ]:
# 부채총계(연결), 자산총계(연결), , 자본총계(연결), 전년도자산총계(연결), 영업활동으로인한현금흐름(연결), 총당기순이익(연결), 유형자산(계)(연결), 매출채권(연결), 총수익(연결), 영업이익(손실)(연결), 매출총이익(손실)(연결)

#############################################################
######################파일 불러오기###########################
#############################################################
file = '이름.csv'
df = pd.read_csv(file, encoding = 'utf-8')


company_job = str(input())

In [63]:
############################################ 테스트 용 ########################################

import pandas as pd

# 예시 데이터프레임 생성
data = {
    '부채총계(연결)': [100],
    '자산총계(연결)': [500],
    '자본총계(연결)': [400],
    '전년도자산총계(연결)': [450],
    '영업활동으로인한현금흐름(연결)': [50],
    '총당기순이익(연결)': [30],
    '유형자산(계)(연결)': [150],
    '매출채권(연결)': [80],
    '매출액(연결)': [700],
    '영업이익(손실)(연결)': [20],
    '매출총이익(손실)(연결)': [60],
    '매출원가(손실)(연결)': [70],
    '유동부채':[70]
}

df = pd.DataFrame(data)

# '부채총계'와 '자산총계'를 포함한 컬럼을 찾기
debt_column = [col for col in df.columns if '부채총계' in col][0]
asset_column = [col for col in df.columns if '자산총계' in col][0]

# 해당 컬럼들의 시리즈를 들고와서 계산
debt_series = df[debt_column]
asset_series = df[asset_column]

# 부채총계 시리즈 / 자산총계 시리즈 계산
ratio_series = debt_series / asset_series
pd.DataFrame(ratio_series)
print(pd.DataFrame(ratio_series))

     0
0  0.2


In [78]:
#############################################################################################################################
############################################### 필요 라이브러리 불러오기 ######################################################
#############################################################################################################################

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import openai
import os

import warnings
warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

# 시각화 한글폰트 설정
plt.rc('font', family='Malgun')
sns.set(font="Malgun Gothic",
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.
import joblib
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Flatten, SimpleRNN, LSTM, GRU
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
import joblib

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

#############################################################################################################################
############################################### 시계열 모델 형식 만들기 #######################################################
#############################################################################################################################

def make_dataframe(df, target_col):
    period_length = 9                            # 2015 ~ 2024
    num_sets = len(df) // period_length
    frames = []
    for i in range(num_sets):
        sub_df = df.iloc[i * period_length : (i + 1) * period_length].reset_index(drop=True)
        sub_df['target_col'] = sub_df['target_col'].shift(-1)
        sub_df = sub_df.dropna().reset_index(drop=True)
        frames.append(sub_df)
    result_df = pd.concat(frames, ignore_index=True)
    return result_df

#############################################################################################################################
########################################### 시계열 데이터 형식 만들기 #########################################################
#############################################################################################################################

def temporalize(x, timesteps):
    nfeature = x.shape[1]
    output_x = []
    for i in range(len(x) - timesteps + 1):
        t = []
        for j in range(timesteps):
            t.append(x[[(i + j)], :])
        output_x.append(t)
    return np.array(output_x).reshape(-1, timesteps, nfeature)

#############################################################################################################################
############################### 첫 데이터 프레임에서 우리가 쓸 데이터 프레임으로 변형  ###########################################
#############################################################################################################################

def make_DataFrame(df, col_name):
    
    col_name = ['DEBT', 'AGR', 'OCF', 'ROA', 'PPE', 'ART', 'ATR', 'OPM', 'GPM', 'ROE', 'NPM']
    
    for col in col_name:
        if col == 'DEBT':
            fir_column = [col for col in df.columns if '부채총계' in col][0]
            sed_column = [col for col in df.columns if '자산총계' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = fir_series / sed_series
            df1 = pd.DataFrame(ratio_series)
            df1.columns = [f'2023_{col}']
        elif col == 'AGR':
            fir_column = [col for col in df.columns if '전년도자산총계' in col][0]
            sed_column = [col for col in df.columns if '자산총계' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = (sed_series - fir_series) / fir_series
            df1[f'2023_{col}'] = ratio_series
        elif col == 'OCF':
            fir_column = [col for col in df.columns if '현금흐름' in col][0]
            sed_column = [col for col in df.columns if '자산총계' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = fir_series / sed_series
            df1[f'2023_{col}'] = ratio_series
        elif col == 'ROA':
            fir_column = [col for col in df.columns if '총당기순이익' in col][0]
            sed_column = [col for col in df.columns if '자산총계' in col][0]
            thr_column = [col for col in df.columns if '전년도자산총계' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            thr_series = df[thr_column]
            ratio_series = fir_series / ((sed_series + thr_series) / 2)
            df1[f'2023_{col}'] = ratio_series
        elif col == 'PPE':
            fir_column = [col for col in df.columns if '유형자산' in col][0]
            sed_column = [col for col in df.columns if '자산총계' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = fir_series / sed_series
            df1[f'2023_{col}'] = ratio_series
        elif col == 'ART':
            fir_column = [col for col in df.columns if '채권' in col][0]
            sed_column = [col for col in df.columns if '매출액' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = fir_series / sed_series
            df1[f'2023_{col}'] = ratio_series
        elif col == 'ATR':
            fir_column = [col for col in df.columns if '매출액' in col][0]
            sed_column = [col for col in df.columns if '자산총계' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = fir_series / sed_series
            df1[f'2023_{col}'] = ratio_series
        elif col == 'OPM':
            fir_column = [col for col in df.columns if '영업이익' in col][0]
            sed_column = [col for col in df.columns if '매출액' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = fir_series / sed_series
            df1[f'2023_{col}'] = ratio_series
        elif col == 'GPM':
            fir_column = [col for col in df.columns if '매출총이익' in col][0]
            sed_column = [col for col in df.columns if '매출액' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = fir_series / sed_series
            df1[f'2023_{col}'] = ratio_series
        elif col == 'ROE':
            fir_column = [col for col in df.columns if '총당기순이익' in col][0]
            sed_column = [col for col in df.columns if '자본총계' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = fir_series / sed_series
            df1[f'2023_{col}'] = ratio_series
        elif col == 'NPM':
            fir_column = [col for col in df.columns if '총당기순이익' in col][0]
            sed_column = [col for col in df.columns if '매출액' in col][0]
            fir_series = df[fir_column]
            sed_series = df[sed_column]
            ratio_series = fir_series / sed_series
            df1[f'2023_{col}'] = ratio_series
        
    df1['Year'] = 2023
    df = df / 10000000000000
    df['Year'] = 2023
    
#####################################################################################################################################
######################################### 타깃(오각형) 마다 모델을 돌려 예측값 생성  ###################################################
#####################################################################################################################################

    timestep = 1 # 타임 스템 
    target_col = ['DEBT', 'AGR', 'ROA', 'PPE', 'ATR']

    #'DEBT', 'AGR', 'ROA', 'PPE', 'ATR'
    for col1 in target_col:
        if col1 == 'DEBT':
            target = f'2023_{col1}'
            x_val = df1.drop(target, axis = 1) 
            scaler_x_debt = joblib.load('scaler_x(DEBT) (1).pkl')
            x_val = scaler_x_debt.transform(x_val)
            x_val = np.array(x_val)
            x_val = temporalize(x_val, timestep)
            pred_debt = load_model('DEBT모델.keras').predict(x_val)
        elif col1 == 'AGR':
            target = f'2023_{col1}'
            x_val = df1.drop(target, axis = 1)
            scaler_x_agr = joblib.load('scaler_x(AGR) (1).pkl')
            x_val = scaler_x_agr.transform(x_val)
            x_val = np.array(x_val)
            x_val = temporalize(x_val, timestep)
            pred_agr = load_model('AGR모델.keras').predict(x_val)
        elif col1 == 'ROA':
            target = f'2023_{col1}'
            x_val = df1.drop(target, axis = 1)
            scaler_x_roa = joblib.load('scaler_x(ROA) (1).pkl')
            x_val = scaler_x_roa.transform(x_val)
            x_val = np.array(x_val)
            x_val = temporalize(x_val, timestep)
            pred_roa = load_model('ROA모델.keras').predict(x_val)
        elif col1 == 'PPE':
            target = f'2023_{col1}'
            x_val = df1.drop(target, axis = 1)
            scaler_x_ppe = joblib.load('scaler_x(PPE) (1).pkl')
            x_val = scaler_x_ppe.transform(x_val)
            x_val = np.array(x_val)
            x_val = temporalize(x_val, timestep)
            pred_ppe = load_model('PPE모델.keras').predict(x_val)
        elif col1 == 'ATR':
            target = f'2023_{col1}'
            x_val = df1.drop(target, axis = 1)
            scaler_x_atr = joblib.load('scaler_x(ATR) (1).pkl')
            x_val = scaler_x_atr.transform(x_val)
            x_val = np.array(x_val)
            x_val = temporalize(x_val, timestep)
            pred_atr = load_model('ATR모델.keras').predict(x_val)
            
            
    df1['2024_DEBT'] = pred_debt
    df1['2024_AGR'] = pred_agr
    df1['2024_ROA'] = pred_roa
    df1['2024_PPE'] = pred_ppe
    df1['2024_ATR'] = pred_atr
    
    df_copy = df1
    
#####################################################################################################################################
######################################### 전년도 비교 (4개) 모델을 돌려 예측값 생성  ###################################################
#####################################################################################################################################

    previous_target_col = ['매출액', '총당기순이익', '자산총계', '부채총계']
    df = df.drop([col for col in df.columns if '전년도자산총계' in col], axis = 1)

    for col2 in previous_target_col:
        if col2 == '매출액':
            x_val2 = df.drop([col for col in df.columns if f'{col2}' in col], axis = 1)
            scaler_x = joblib.load('scaler_x(총수익) (1).pkl')
            x_val2 = scaler_x.transform(x_val2)
            x_val2 = np.array(x_val2)
            x_val2 = temporalize(x_val2, timestep)
            pred_mae = load_model('총수익모델 (1).keras').predict(x_val2)
        elif col2 == '부채총계':
            x_val2 = df.drop([col for col in df.columns if f'{col2}' in col], axis = 1)
            scaler_x = joblib.load('scaler_x(부채총계) (1).pkl')
            x_val2 = scaler_x.transform(x_val2)
            x_val2 = np.array(x_val2)
            x_val2 = temporalize(x_val2, timestep)
            pred_bu = load_model('부채총계모델 (1).keras').predict(x_val2)
        elif col2 == '총당기순이익':
            x_val2 = df.drop([col for col in df.columns if f'{col2}' in col], axis = 1)
            scaler_x = joblib.load('scaler_x(당기순이익) (1).pkl')
            x_val2 = scaler_x.transform(x_val2)
            x_val2 = np.array(x_val2)
            x_val2 = temporalize(x_val2, timestep)
            pred_dang = load_model('당기순이익모델 (1).keras').predict(x_val2)
        elif col2 == '자산총계':
            x_val2 = df.drop([col for col in df.columns if f'{col2}' in col], axis = 1)            
            scaler_x = joblib.load('scaler_x(자산총계) (1).pkl')
            x_val2 = scaler_x.transform(x_val2)
            x_val2 = np.array(x_val2)
            x_val2 = temporalize(x_val2, timestep)
            pred_jasan = load_model('자산총계모델 (1).keras').predict(x_val2)

    df1['2023_당기순이익'] = df[[col for col in df.columns if '당기순이익' in col]]
    df1['2023_자산총계'] = df[[col for col in df.columns if '자산총계' in col]]
    df1['2023_부채총계'] = df[[col for col in df.columns if '부채총계' in col]]
    df1['2023_매출액'] = df[[col for col in df.columns if '매출액' in col]]
    
    df1['2024_당기순이익예측'] = pred_dang
    df1['2024_자산총계예측'] = pred_jasan
    df1['2024_부채총계예측'] = pred_bu
    df1['2024_매출액예측'] = pred_mae
    
    return df1
    
####################################################################################################################################
############################################### 생성형 AI 모델 및 LangChain 함수 설정  ###############################################
####################################################################################################################################



In [77]:
make_DataFrame(df, col_name)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 440ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 432ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 998ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 483ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 429ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 400ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step


,2023_DEBT,2023_AGR,2023_OCF,2023_ROA,2023_PPE,2023_ART,2023_ATR,2023_OPM,2023_GPM,2023_ROE,...,2024_PPE,2024_ATR,2023_당기순이익,2023_자산총계,2023_부채총계,2023_매출액,2024_당기순이익예측,2024_자산총계예측,2024_부채총계예측,2024_매출액예측
0,0.2,0.111111,0.1,0.063158,0.3,0.114286,1.4,0.028571,0.085714,0.075,...,0.255201,1.062618,3.000000e-12,5.000000e-11,1.000000e-11,7.000000e-11,-0.003561,-0.013024,0.002837,-0.01819
